In [1]:
def YuleSimonProcess(alpha=0.1,Ts=(1,100),initial_Tagset=[[0]],new_tagset="",how_new_tag="enumerate",from_num_new_tag=0):
    import numpy as np
    import random 
    #alpha:float of novelty rate or list of each post's novelty rate or function of novelty rate of each timestep[1..N].:default=0.03
    #Ts:(int,int) or list of number of Tag each post(and number of post).Default=(1,100)
    #initial_Tagset: list of list of new_tag:default: [[0]]
    #new_tagset:str or list of new tag set,if str or there is no non-used new-tag, followed how_new_tag: default:na
    #how_new_tag:"enumerate","str_enumerate" or else: if enumerate, this algorith give order number else, stop making a new_tag.
    #from_num_new_tag:int of starting point of enumerate of new tag :default:from_num_new_tag=0
    if type(initial_Tagset)==list:
        if len(initial_Tagset)>0:
            tagsets=initial_Tagset
            if type(initial_Tagset[0])==list:
                vase_tag=[]
                for ts in tagsets:
                    if type(ts)!=list:
                        raise ValueError("If Ts is list, type of all value is needed to list of tagset.")
                    vase_tag+=ts
            else:
                vase_tag=tagset
        else:
            raise ValueError("initail_Tagset list is NAN, you need to initial_Tagset length is higher than 0.")
    else:
        raise ValueError("initail_Tagset need to list:"+type(initial_Tagset))
    if type(Ts)==tuple or type(Ts)==list:
        if type(Ts)==tuple:
            if type(Ts[0])==int and type(Ts[1])==int:
                N=Ts[1]
                Ts=[Ts[0]]*N
            else:
                raise ValueError("tupple of Ts is need to (int,int):("+type(Ts[0])+","+type(Ts[1])+")")
        else:
            if len(Ts)>0:
                N=len(N)
            else:
                raise ValueError("Need to length of list of Ts is higher than 0.")
    else:
        raise ValueError("Ts need to tuple of (num of tag each post, num of post) or list of num of tag.:"+type(Ts))
    if type(alpha)==float or type(alpha)==list or type(alpha)==type(lambda x:x):
        if type(alpha)==list:
            if len(alpha)!=N:
                raise ValueError("different length between alpha and Ts:len(alpha)="+str(len(a))+",length of T="+str(len(N)))
        elif type(alpha)==float:
            alpha=[alpha]*N
        else:
            alpha=[alpha(t) for t in range(1,N+1)]
    else:
        if type(alpha)==int:
            alpha=[float(alpha)]*N
        else:
            raise ValueError("alpha need to list, float or function of novelty rate.:"+type(a))
    
    if type(new_tagset)==str or type(tagset)==list:
        pass
    else:
        raise ValueError("new_tagset need to str(something OK) or list of new tag.:"+type(new_tagset))
    if type(how_new_tag)==str:
        if how_new_tag == "enumerate":
            create_flag=1
        elif how_new_tag == "str_enumerate":
            create_flag=2
        else:
            if type(new_tagset)!=list:
                print("Warning:You set non-'enumurate' value of how_new_tag, but you choice the new_tagset is str. The result is produced by only initial_Tagset.")
            create_flag=0
            
    else:
        raise ValueError("Type of how_new_tag is needed to str:"+type(how_new_tag))
    if type(from_num_new_tag)==int:
        while(from_num_new_tag in vase_tag):
            #print("Warning:There is number of from_num_new_tag in initilal_Tagset.Increaseing it")
            from_num_new_tag+=1
            
    else:
        raise ValueError("Type of from_num_new_tag is wrong.You should be int.")
    new_tag_id=from_num_new_tag
    def create_tag():
        nonlocal new_tagset
        nonlocal vase_tag
        nonlocal new_tag_id
        if type(new_tagset)==str:
            if create_flag==2:
                tag=str(new_tag_id)
            else:
                tag=new_tag_id
            new_tag_id+=1
        else:
            tag=new_tagset.pop(0)
        vase_tag.append(tag)
        return tag
    def PA():
        nonlocal vase_tag
        tag=random.choice(vase_tag)
        vase_tag.append(tag)
        return tag
    def YSP(a):
        if create_flag==0 and len(new_tagset)==0:
            a=0
        p=random.uniform(0, 1)
        if p<a:
            return create_tag()
        else:
            return PA()
    for i in range(N):
        tagset=[YSP(alpha[i]) for _ in range(Ts[i])]
        tagsets.append(tagset)
    return tagsets

In [2]:
YuleSimonProcess()

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [2],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [3],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [4],
 [1],
 [0],
 [5],
 [0],
 [0],
 [0],
 [6],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [2],
 [0],
 [0],
 [5],
 [6],
 [0],
 [0],
 [1],
 [0],
 [6],
 [2],
 [0],
 [0],
 [0],
 [7],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [6],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [2]]

In [3]:
YuleSimonProcess(alpha=lambda x:1/(x*0.7)+0.02,Ts=(3,100),initial_Tagset=[[0,1,2]])

[[0, 1, 2],
 [3, 4, 5],
 [6, 4, 4],
 [2, 2, 2],
 [2, 3, 2],
 [0, 3, 2],
 [2, 7, 4],
 [2, 2, 8],
 [9, 4, 9],
 [0, 4, 4],
 [2, 2, 2],
 [8, 4, 3],
 [3, 2, 2],
 [2, 4, 2],
 [3, 2, 4],
 [2, 10, 11],
 [11, 4, 12],
 [4, 4, 4],
 [4, 4, 2],
 [0, 8, 2],
 [4, 2, 0],
 [8, 4, 4],
 [4, 2, 2],
 [12, 4, 4],
 [2, 2, 2],
 [1, 4, 4],
 [4, 13, 14],
 [15, 15, 2],
 [8, 4, 4],
 [5, 4, 6],
 [2, 3, 2],
 [2, 11, 4],
 [3, 0, 11],
 [2, 2, 1],
 [4, 16, 2],
 [2, 2, 7],
 [16, 4, 7],
 [2, 15, 2],
 [16, 3, 4],
 [1, 2, 3],
 [2, 0, 6],
 [2, 2, 15],
 [3, 11, 17],
 [18, 2, 15],
 [8, 7, 2],
 [0, 2, 19],
 [2, 19, 2],
 [0, 19, 2],
 [3, 4, 8],
 [2, 2, 8],
 [3, 2, 0],
 [0, 2, 2],
 [2, 14, 3],
 [2, 19, 4],
 [7, 5, 8],
 [0, 3, 0],
 [4, 18, 19],
 [4, 3, 4],
 [0, 2, 4],
 [19, 12, 19],
 [19, 1, 2],
 [2, 2, 8],
 [4, 2, 2],
 [4, 2, 2],
 [2, 3, 9],
 [2, 0, 4],
 [3, 2, 2],
 [11, 2, 4],
 [1, 0, 6],
 [2, 4, 8],
 [2, 0, 2],
 [2, 4, 15],
 [2, 2, 3],
 [20, 4, 3],
 [9, 0, 2],
 [4, 19, 0],
 [4, 1, 2],
 [2, 2, 4],
 [2, 1, 4],
 [4, 4, 0],
 [4, 